In [3]:
# !pip install llama-index
# !pip install llama-index-embeddings-mistralai
# !pip install llama-index-llms-mistralai
# !pip install python-dotenv

# Imports 

In [25]:
import os
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import SummaryIndex, VectorStoreIndex
from llama_index.core import Settings
from llama_index.embeddings.mistralai import MistralAIEmbedding
from llama_index.llms.mistralai import MistralAI
from llama_index.core.tools import QueryEngineTool

 # Load the PDF into LlamaIndex

In [2]:
# Load the document
documents = SimpleDirectoryReader(input_files=["An_Overview_of_Artificial_Intelligence_Ethics.pdf"]).load_data()

# Display the first document
print(documents[0])

Doc ID: bcaa3acb-42b1-4e20-90ff-96c911d1bcf0
Text: IEEE TRANSACTIONS ON ARTIFICIAL INTELLIGENCE, VOL. 4, NO. 4,
AUGUST 2023 799 An Overview of Artiﬁcial Intelligence Ethics Changwu
Huang , Member, IEEE, Zeqi Zhang, Bifei Mao, and Xin Yao, Fellow, IEEE
Abstract—Artiﬁcial intelligence (AI) has profoundly changed and will
continue to change our lives. AI is being applied in more and more
ﬁelds and ...


# Split the Text Using SentenceSplitter

In [11]:
# Initialize the splitter with a chunk size of 512 characters
splitter = SentenceSplitter(chunk_size=512)

# Split the document into chunks (nodes)
nodes = splitter.get_nodes_from_documents(documents, show_progress=True)

# Display the first chunk
print(nodes[0])

Parsing nodes:   0%|          | 0/21 [00:00<?, ?it/s]

Node ID: 5e6fdd93-888e-4ca9-9e5b-fb77db04f91c
Text: IEEE TRANSACTIONS ON ARTIFICIAL INTELLIGENCE, VOL. 4, NO. 4,
AUGUST 2023 799 An Overview of Artiﬁcial Intelligence Ethics Changwu
Huang , Member, IEEE, Zeqi Zhang, Bifei Mao, and Xin Yao, Fellow, IEEE
Abstract—Artiﬁcial intelligence (AI) has profoundly changed and will
continue to change our lives. AI is being applied in more and more
ﬁelds and ...


#### document loading and text splitting steps are correctly executed. The "Error displaying widget" message is just a Jupyter Notebook issue and does not affect the correctness of the output. 

# Set Up Mistral AI API Key

In [4]:
os.environ["MISTRAL_API_KEY"] = "EQGOZOmRj95QxXpOJUI0yOMjnzTF1ym5"

Settings.llm = MistralAI(api_key=os.getenv("MISTRAL_API_KEY"))
Settings.embed_model = MistralAIEmbedding(model_name="mistral-embed", api_key=os.getenv("MISTRAL_API_KEY"))

# Create a Summary Index and Vector Index

In [5]:
# Create a summary index
summary_index = SummaryIndex(nodes[:10])

# Create a vector index
vector_index = VectorStoreIndex(nodes[:10])

print("Indexes created successfully with Mistral AI!")

Indexes created successfully with Mistral AI!


# Set Up Query Engines

In [27]:
# Define the query engine for summarization
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)

# Define the query engine for vector search
vector_query_engine = vector_index.as_query_engine()

print("Query engines set up successfully!")

Query engines set up successfully!


In [29]:
# Define a tool for summarization
summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description="Useful for summarizing the document.",
)

# Define a tool for retrieving specific details
vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description="Useful for retrieving specific details from the document.",
)

print("Query tools created successfully!")

Query tools created successfully!


# Test the Query System

In [32]:
response = summary_query_engine.query("Summarize this document.")
print(response)

The document provides a comprehensive overview of the field of Artificial Intelligence (AI) ethics. It begins by highlighting the profound impact of AI on various sectors such as autonomous driving, medical care, finance, and internet services, and the consequent ethical concerns that arise, including privacy leakage, discrimination, unemployment, and security risks. The document emphasizes the importance of AI ethics as a research topic and a concern for individuals, organizations, and society at large.

The article aims to offer a systematic and comprehensive overview of AI ethics by summarizing and analyzing ethical risks and issues, ethical guidelines and principles issued by different organizations, approaches for addressing ethical issues in AI, and methods for evaluating the ethics of AI. It also points out challenges in implementing ethics in AI and provides future perspectives.

The main contributions of the article include a comprehensive overview of AI ethics, a new categori

In [34]:
response = vector_query_engine.query("What are the key challenges discussed in this paper?")
print(response)

The key challenges discussed include implementing ethics in AI and designing ethical AI. Additionally, the paper points out some future perspectives that could guide further research in the AI ethics field.


In [36]:
response = vector_query_engine.query("How does this paper define fairness in AI?")
print(response)

The paper does not explicitly define fairness in AI.
